# Model #4 for image prediction

In [22]:
import numpy as np
import pandas as pd
# from skimage import imread
from matplotlib.image import imread
import plotly.express as px
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models,callbacks,metrics,backend
from tensorflow.keras.optimizers import RMSprop
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# sys.path.append(os.path.abspath(r'C:\Users\raulg\Documents\THEBRIDGE_DS\0.-Repo_Git\ml_alzheimer_class\src'))
sys.path.append(os.path.relpath('../src'))
import utils as ut
import math


This model looks for pure accuracy optimization. Three ways for this:
- 4a adds a new layer for the CNN model.
- 4b uses a different optimizer.
- 4c takes a resized data of 64x64 (instead of 32x32)

## Data loading and mix

In [3]:
x1_train,x1_test,x1_val,y1_train,y1_test,y1_val = ut.img_images_load(val_set = True)


x1_train shape (34326, 32, 32, 1)
x1_test shape (6058, 32, 32, 1)
y1_train shape (34326,)
y1_test shape (6058,)
y1_train distribution: 
 [[    0 10880]
 [    1  9520]
 [    2  8377]
 [    3  5549]]
y1_test distribution: 
 [[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]
x1_val shape (6866, 32, 32, 1)
y1_val shape (6866,)
y1_val distribution: 
 [[   0 2176]
 [   1 1904]
 [   2 1676]
 [   3 1110]]


## Model 4a

In [4]:
model4a = models.Sequential()
model4a.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model4a.add(layers.MaxPooling2D((2, 2)))
model4a.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4a.add(layers.MaxPooling2D((2, 2)))
model4a.add(layers.Conv2D(32, (3, 3), activation='relu'))
model4a.add(layers.MaxPooling2D((2, 2)))
model4a.add(layers.Flatten())

# Capa densa con Dropout
model4a.add(layers.Dense(32, activation='relu'))
model4a.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model4a.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model4a.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', #sparse_categorical_crossentropy
              metrics=['accuracy'])

model4a.fit(x1_train,y1_train,batch_size=128,epochs=70,validation_data=(x1_val,y1_val), callbacks=callbacks.EarlyStopping(patience=5)
          )

Epoch 1/70


c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.3364 - loss: 1.3286 - val_accuracy: 0.5386 - val_loss: 1.0659
Epoch 2/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.5206 - loss: 1.0245 - val_accuracy: 0.6175 - val_loss: 0.8284
Epoch 3/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5847 - loss: 0.8809 - val_accuracy: 0.6484 - val_loss: 0.7468
Epoch 4/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.6167 - loss: 0.8015 - val_accuracy: 0.6689 - val_loss: 0.7166
Epoch 5/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.6497 - loss: 0.7385 - val_accuracy: 0.6858 - val_loss: 0.6849
Epoch 6/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.6774 - loss: 0.6971 - val_accuracy: 0.7192 - val_loss: 0.6127
Epoch 7/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7003 - loss: 0.6399 - val_accuracy: 0.7310 - val_loss: 0.5867
Epoch 8/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7186 - loss: 0.6071 - val_accuracy: 0.726

In [6]:
model4a.evaluate(x1_test, y1_test)

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8424 - loss: 0.4205


[0.4229222536087036, 0.8445031642913818]

In [8]:
recall_score(y1_test, model4a.predict(x1_test).argmax(axis=1),average='weighted')

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


np.float64(0.84450313634863)

In [14]:
confusion_matrix(y1_test, model4a.predict(x1_test).argmax(axis=1),normalize='pred') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[0.83183633, 0.12248521, 0.02926116, 0.00601805],
       [0.13872255, 0.75621302, 0.08339429, 0.01003009],
       [0.02944112, 0.11360947, 0.88441843, 0.01905717],
       [0.        , 0.00769231, 0.00292612, 0.96489468]])

## Model 4b

In [15]:
model4b = models.Sequential()
model4b.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model4b.add(layers.MaxPooling2D((2, 2)))
model4b.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4b.add(layers.MaxPooling2D((2, 2)))
model4b.add(layers.Conv2D(32, (3, 3), activation='relu'))
model4b.add(layers.MaxPooling2D((2, 2)))
model4b.add(layers.Flatten())

# Capa densa con Dropout
model4b.add(layers.Dense(32, activation='relu'))
model4b.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model4b.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model4b.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', #sparse_categorical_crossentropy
              metrics=['accuracy'])

model4b.fit(x1_train,y1_train,batch_size=128,epochs=70,validation_data=(x1_val,y1_val), callbacks=callbacks.EarlyStopping(patience=5)
          )

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.3555 - loss: 1.3118 - val_accuracy: 0.5334 - val_loss: 1.0328
Epoch 2/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.5192 - loss: 1.0215 - val_accuracy: 0.6148 - val_loss: 0.8327
Epoch 3/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.5886 - loss: 0.8726 - val_accuracy: 0.6306 - val_loss: 0.7515
Epoch 4/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.6308 - loss: 0.7804 - val_accuracy: 0.6904 - val_loss: 0.6913
Epoch 5/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.6712 - loss: 0.7099 - val_accuracy: 0.6882 - val_loss: 0.6615
Epoch 6/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7062 - loss: 0.6517 - val_accuracy: 0.6564 - val_loss: 0.6855
Epoch 7/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.7310 - loss: 0.6010 - val_accuracy: 0.7693 - val_loss: 0.5385
Epoch 8/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7589 - loss: 0.5439 - val_accu

In [16]:
model4b.evaluate(x1_test, y1_test)

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8776 - loss: 0.4065


[0.41098645329475403, 0.8763618469238281]

In [17]:
recall_score(y1_test, model4b.predict(x1_test).argmax(axis=1),average='weighted')

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


np.float64(0.8763618355893034)

In [18]:
confusion_matrix(y1_test, model4b.predict(x1_test).argmax(axis=1),normalize='pred') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[0.8940926 , 0.10171496, 0.04379085, 0.00104384],
       [0.08408728, 0.81431106, 0.08954248, 0.00835073],
       [0.02075572, 0.07510349, 0.85490196, 0.00521921],
       [0.0010644 , 0.00887049, 0.01176471, 0.98538622]])

 ## Model 4c

### Resizing to 64x64

In [59]:
ROOT_PATH = os.getcwd()
TRAIN_PATH = "../data/images/train/"
TEST_PATH = "../data/images/val/"
### Training dataframe
main_dir = TRAIN_PATH
folders = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented']
img_dict = {'image':[],'label':[]}

for folder in folders:
    folder_path = os.path.join(main_dir,folder)
    files_folder = os.listdir(folder_path)
    for file in files_folder:
        file_name = os.path.basename(file)
        img_dict['image'].append(file_name)
        img_dict['label'].append(folder)

df_train = pd.DataFrame(img_dict)
df_train.head()

,image,label
0,00046ff7-0fae-4796-9a2b-2df47095bfe6.jpg,MildDemented
1,000cdcc4-3e54-4034-a538-203c8047b564.jpg,MildDemented
2,0013ad52-8398-4892-9fde-060eb1ae956d.jpg,MildDemented
3,0015bdfc-2059-4195-8eff-9c5cf2e2c025.jpg,MildDemented
4,001f15e8-6ad4-4e7a-bc36-b62b984dad69.jpg,MildDemented


In [9]:
x1 = []
y1 = []

for i,row in df_train.iterrows():
    count = 1
    print(f'Opening image #{count} ,{row['label']}')
    image_path = os.path.join(TRAIN_PATH, row['label'],row['image'])
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        continue

    image = cv2.resize(image, (64, 64)) ### 64x64 pixels
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) ### Conversion to gray scale
    x1.append(image)
    y1.append(row['label'])
    count +=1

### Lists
x1 = np.array(x1)
y1 = np.array(y1)

Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening 

(33984, 64, 64)
(33984,)


In [21]:
pickle.dump(x1,open('../data/images/processed_train/x1_64.pkl','wb'))
pickle.dump(y1,open('../data/images/processed_train/y1_64.pkl','wb'))

In [60]:
### Val dataframe
main_dir = TEST_PATH
folders = ['MildDemented','ModerateDemented','NonDemented','VeryMildDemented']
img_dict = {'image':[],'label':[]}

for folder in folders:
    folder_path = os.path.join(main_dir,folder)
    files_folder = os.listdir(folder_path)
    for file in files_folder:
        file_name = os.path.basename(file)
        img_dict['image'].append(file_name)
        img_dict['label'].append(folder)

df_test = pd.DataFrame(img_dict)
df_test.head()

,image,label
0,26 (19).jpg,MildDemented
1,26 (20).jpg,MildDemented
2,26 (21).jpg,MildDemented
3,26 (22).jpg,MildDemented
4,26 (23).jpg,MildDemented


In [5]:
x2 = []
y2 = []

for i,row in df_test.iterrows():
    count = 1
    print(f'Opening image #{count} ,{row['label']}')
    image_path = os.path.join(TEST_PATH, row['label'],row['image'])
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        continue

    image = cv2.resize(image, (64, 64)) ### 64x64 pixels
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) ### Conversion to gray scale
    x2.append(image)
    y2.append(row['label'])
    count +=1

### Lists
x2 = np.array(x2)
y2 = np.array(y2)

Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening image #1 ,MildDemented
Opening 

In [3]:
pickle.dump(x2,open('../data/images/processed_val/x2_64.pkl','wb'))
pickle.dump(y2,open('../data/images/processed_val/y2_64.pkl','wb'))

NameError: name 'x2' is not defined

Data conversion with img_images_load

In [46]:
x1_train,x1_test,x1_val,y1_train,y1_test,y1_val = ut.img_images_load('../data/images/processed_train/x1_64.pkl','../data/images/processed_train/y1_64.pkl',
                                                                     '../data/images/processed_val/x2_64.pkl','../data/images/processed_val/y2_64.pkl',
                                                                     val_set = True,reshuffle=True,img_size=64)
# x1_train,x1_test,x1_val,y1_train,y1_test,y1_val = ut.img_images_load(val_set = True,reshuffle=False)

x1_train shape (34326, 64, 64, 1)
x1_test shape (6058, 64, 64, 1)
y1_train shape (34326,)
y1_test shape (6058,)
y1_train distribution: 
 [[    0 10880]
 [    1  9520]
 [    2  8377]
 [    3  5549]]
y1_test distribution: 
 [[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]
x1_val shape (6866, 64, 64, 1)
y1_val shape (6866,)
y1_val distribution: 
 [[   0 2176]
 [   1 1904]
 [   2 1676]
 [   3 1110]]


### Modelling

In [ ]:
### COMP TIME: 14 minutes
model4c = models.Sequential()
model4c.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 1)))
model4c.add(layers.MaxPooling2D((2, 2)))
model4c.add(layers.Conv2D(128, (3, 3), activation='relu'))
model4c.add(layers.MaxPooling2D((2, 2)))
model4c.add(layers.Conv2D(64, (3, 3), activation='relu'))
model4c.add(layers.MaxPooling2D((2, 2)))
model4c.add(layers.Flatten())

# Capa densa con Dropout
model4c.add(layers.Dense(64, activation='relu'))
model4c.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model4c.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model4c.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', #sparse_categorical_crossentropy
              metrics=['accuracy'])

model4c.fit(x1_train,y1_train,batch_size=128,epochs=70,validation_data=(x1_val,y1_val), callbacks=callbacks.EarlyStopping(patience=5)
          )

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 53s 60ms/step - accuracy: 0.4460 - loss: 1.1656 - val_accuracy: 0.6615 - val_loss: 0.7004
Epoch 2/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 49s 57ms/step - accuracy: 0.7088 - loss: 0.6585 - val_accuracy: 0.7410 - val_loss: 0.5694
Epoch 3/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 51s 59ms/step - accuracy: 0.8091 - loss: 0.4627 - val_accuracy: 0.8357 - val_loss: 0.3756
Epoch 4/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 52s 60ms/step - accuracy: 0.8592 - loss: 0.3550 - val_accuracy: 0.8465 - val_loss: 0.3885
Epoch 5/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 54s 63ms/step - accuracy: 0.8876 - loss: 0.2919 - val_accuracy: 0.7694 - val_loss: 0.4790
Epoch 6/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 53s 61ms/step - accuracy: 0.9015 - loss: 0.2568 - val_accuracy: 0.9010 - val_loss: 0.2832
Epoch 7/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 51s 60ms/step - accuracy: 0.9190 - loss: 0.2158 - val_accuracy: 0.8842 - val_loss: 0.3618
Epoch 8/70
859/859 ━━━━━━━━━━━━━━━━━━━━ 49s 57ms/step - accuracy: 0.9262 - loss: 0.2058 - 

In [22]:
model4c.evaluate(x1_test, y1_test)

190/190 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9016 - loss: 0.3264


[0.3425421416759491, 0.9045889973640442]

In [23]:
recall_score(y1_test, model4c.predict(x1_test).argmax(axis=1),average='weighted')

190/190 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step


np.float64(0.9045889732585012)

In [24]:
confusion_matrix(y1_test, model4c.predict(x1_test).argmax(axis=1)) #,normalize='pred'

  1/190 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step

190/190 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step


array([[1686,  185,   46,    3],
       [ 105, 1468,   95,   12],
       [  29,   83, 1353,   14],
       [   0,    3,    3,  973]])

In [2]:
# model4c.save("../models/image/model_4.keras")
model4c = models.load_model("../models/image/model_4.keras")

In [13]:
x1_test.shape

(6058, 64, 64, 1)

### Testing for Streamlit

In [14]:
test_pred = model4c.predict(x1_test)
test_pred[1]

190/190 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step


array([3.2050171e-05, 1.3295245e-01, 6.8128414e-02, 7.9888707e-01],
      dtype=float32)

In [23]:
test_pred[1].round(3)

array([0.   , 0.133, 0.068, 0.799], dtype=float32)

In [35]:
test_pred_show = test_pred[1].round(3)
mapping = {
        'Non Demented': 0,
        'Very Mild Demented': 1,
        'Mild Demented': 2,
        'Moderate Demented': 3
    }

In [40]:
test_pred_show

array([0.   , 0.133, 0.068, 0.799], dtype=float32)

In [33]:
mapping.keys()

dict_keys(['NonDemented', 'VeryMildDemented', 'MildDemented', 'ModerateDemented'])

In [ ]:
px.pie(values=test_pred_show,names=mapping.keys(), title='Probabilities of the prediction')

In [42]:
test = np.array([[0., 0., 0., 1.]])
test.shape

(1, 4)

### Checking val images

In [43]:
x4_test = pickle.load(open('../data/images/processed_val/x2.pkl','rb'))
y4_test = pickle.load(open('../data/images/processed_val/y2.pkl','rb'))
x4_test

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [48]:
x1_train.shape

(27460, 64, 64, 1)

In [50]:
x1_train,x1_test,y1_train,y1_test = ut.img_images_load('../data/images/processed_train/x1_64.pkl','../data/images/processed_train/y1_64.pkl',
                                                                     '../data/images/processed_val/x2_64.pkl','../data/images/processed_val/y2_64.pkl',
                                                                     val_set = False,reshuffle=False,img_size=64)

In [ ]:
test_pred = model4c.predict(x1_test)


200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step


array([[5.4636247e-15, 1.2052979e-07, 9.9999988e-01, 1.5491558e-22],
       [1.5056772e-11, 5.1605903e-10, 1.0000000e+00, 1.4589005e-20],
       [4.9251745e-09, 3.7856478e-11, 1.0000000e+00, 9.2433256e-19],
       ...,
       [7.1901272e-06, 9.9999273e-01, 1.3238092e-07, 1.7300997e-17],
       [6.1349507e-05, 9.9993861e-01, 1.1504474e-09, 6.4468320e-17],
       [6.0217329e-09, 1.0000000e+00, 2.3761105e-08, 2.7121735e-13]],
      dtype=float32)

In [65]:
df_test_results = pd.DataFrame(test_pred.round(3))
df_test_results = pd.concat([df_test_results,df_test],axis=1)

In [68]:
df_test_results[(df_test_results[0]<1)&(df_test_results[0]>0)&(df_test_results['label']=='NonDemented')]

,0,1,2,3,image,label
964,0.963,0.037,0.0,0.0,26 (65).jpg,NonDemented
967,0.999,0.001,0.0,0.0,26 (68).jpg,NonDemented
970,0.952,0.048,0.0,0.0,26 (71).jpg,NonDemented
971,0.990,0.010,0.0,0.0,26 (72).jpg,NonDemented
972,0.997,0.003,0.0,0.0,26 (73).jpg,NonDemented
...,...,...,...,...,...,...
4123,0.999,0.000,0.0,0.0,nonDem966.jpg,NonDemented
4125,0.991,0.009,0.0,0.0,nonDem968.jpg,NonDemented
4127,0.947,0.053,0.0,0.0,nonDem97.jpg,NonDemented
4139,0.999,0.001,0.0,0.0,nonDem980.jpg,NonDemented


In [ ]:
test_pred = model4c.predict(x1_test)


ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None